In [1]:
import awkward as ak
import numpy as np
import time
import coffea

print(coffea.__version__)
from coffea import util
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
files = [
    "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_170to300_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/0CDD30E0-B953-E54B-BB7A-BFB45BE6EC93.root",
  "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_300to470_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/120000/3788BBD3-3B70-BE48-B01A-4FA395E1E0B6.root",
 "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_470to600_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/F6B6A983-3E2B-7B4D-A2CE-E03AB6E32387.root",
 "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_600to800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/FDF7FBE4-9226-994A-BD45-29995A3CD62F.root",
 "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_800to1000_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/2CEB03D7-5B1A-EE46-9C21-13ED3E349AF4.root",
 "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1000to1400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/DB87A517-0584-1A41-BB18-AAE7C9AAE5B8.root",
 "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1400to1800_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/BFE2E7E9-82CB-FE47-9B43-EBA5C373AF60.root",
 "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_1800to2400_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/5213B536-BAA1-CC43-A58A-B1E41E2680F5.root",
 "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_2400to3200_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/E8400F91-3216-EA46-A9F1-8CC7EF5261B0.root",
 "root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/B21BFDD8-B878-784C-AFB6-5CDE131F5C39.root"
]
#/store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/70000/7CB92FEA-6DFF-7647-9D7A-094D1F84245B.root has timeout error
events = NanoEventsFactory.from_root(files[0], schemaclass=NanoAODSchema).events()

0.7.15


In [2]:
from coffea import processor, hist

#####
##### TO DO #####
# find misses --> need to do deltaR matching by hand --> if no reco miss
# do Rivet routine


class DijetHists(processor.ProcessorABC):
    def __init__(self, ptcut=30., etacut = 2.5):
        # should have separate lower ptcut for gen
        self.ptcut = ptcut
        self.etacut = etacut
        dataset_cat = hist.Cat("dataset", "Dataset")
        jet_cat = hist.Cat("jetNumb", "Jet")
        parton_cat = hist.Cat("partonFlav", "Parton Flavour")
        mass_bin = hist.Bin("mass", "Jet Mass (GeV)", 50, 0, 500.)
        mass_gen_bin = hist.Bin("mass", "Gen Jet Mass (GeV)", 100, 0, 500.)
        pt_bin = hist.Bin("pt", "Jet pT (GeV)", 60, 0, 2400.)
        bdisc_bin = hist.Bin("bdisc", "B-tag discriminator", 10, 0.0, 1.)
        frac_bin = hist.Bin("gfrac", "Gluon fraction", 10, 0.0, 1.)
        eta_bin = hist.Bin("eta", "Eta", 25, -2.5, 2.5)
        self._histos = processor.dict_accumulator({
        'jet_pt':             hist.Hist("Events", jet_cat, parton_cat, pt_bin),
#        'jet_pt_m':           hist.Hist("Events", jet_cat, pt_bin, mass_bin),  #why did I need this hist?
        'jet_eta':            hist.Hist("Events", jet_cat, parton_cat, eta_bin),
        'jet_mass':           hist.Hist("Events", jet_cat, parton_cat, mass_bin),
#        'misses_pt_m':        hist.Hist("Events", pt_bin, mass_gen_bin),
#        'fakes_pt_m':         hist.Hist("Events", pt_bin, mass_bin),
        'jet_response':       hist.Hist("Events", pt_bin, mass_bin, pt_bin, mass_gen_bin),
#        'genjet_pt':          hist.Hist("Events", pt_bin),
#        'genjet_eta':         hist.Hist("Events", eta_bin),
#        'genjet_mass':        hist.Hist("Events", mass_bin),
        'weights':            hist.Hist("Events", dataset_cat, hist.Bin("weights", "gen weights", 60, 0, 1.)),
#         'gluonPurity':      hist.Hist("Events", jet_cat, frac_bin),
        'gluonPurity':        processor.defaultdict_accumulator(int),
        'nGluonJets':         processor.defaultdict_accumulator(int),
        'cutflow':            processor.defaultdict_accumulator(int),
        })
    
    @property
    def accumulator(self):
        return self._histos
    
    # we will receive a NanoEvents instead of a coffea DataFrame
    def process(self, events):
        out = self.accumulator.identity()
        dijetEvents = events[(ak.num(events.FatJet) >= 2) & (ak.num(events.GenJetAK8) >= 2)]
        
#         for i in range(0,10):
#             print("Check that jets and are ordered by pt: ", events.FatJet[i,:].pt, "\n")
#             print("and that gen jets are order by pt:", events.GenJetAK8[i,:].pt, "\n")
        
        #get leading 2 jets
        fatJets = dijetEvents.FatJet[:,0:2]
        genJets = dijetEvents.GenJetAK8[:,0:2]
        
#         print("FatJet fields ", fatJets.fields, "\n")
#         print("GenJet fields ", genJets.fields, "\n")
        
        
        jet1 = dijetEvents.FatJet[:,0]
        jet2 = dijetEvents.FatJet[:,1]
        genjet1 = dijetEvents.GenJetAK8[:,0]
        genjet2 = dijetEvents.GenJetAK8[:,1]
        
        print("Initial # of dijet events ", len(fatJets), " and gen dijet events ", len(genJets), "\n")
        
        #calculate dphi_min
        dphi12 = (np.abs(jet1.delta_phi(jet2)) > 2.)
        dphi12_gen = (np.abs(genjet1.delta_phi(genjet2)) > 2.)
        
        #apply dphi gen and reco selection, pt cuts
        #see CMS PAS SMP-20-010 for selections
        
        
        #want to implement pt > 200GeV to be consistent with dijets?
        pt_cut = ak.all(dijetEvents.FatJet.pt > self.ptcut, axis = -1)
        pt_cut_gen = ak.all(dijetEvents.GenJetAK8.pt > self.ptcut, axis = -1)
        
        eta_cut = ak.all(np.abs(dijetEvents.FatJet.eta) < self.etacut, axis = -1)
        eta_cut_gen = ak.all(np.abs(dijetEvents.GenJetAK8.eta) < self.etacut, axis = -1)
        
        asymm = np.abs(jet1.pt - jet2.pt)/(jet1.pt + jet2.pt)
        asymm_gen  = np.abs(genjet1.pt - genjet2.pt)/(genjet1.pt + genjet2.pt)
        
        dijetEvents = dijetEvents[eta_cut & eta_cut_gen & (asymm < 0.3) & (asymm_gen < 0.3) & 
                                  pt_cut & dphi12 & pt_cut_gen & dphi12_gen]
        
        print("# of dijet events after eta cut", len(dijetEvents.FatJet), " and gen dijet events ", len(dijetEvents.GenJetAK8), "\n")
                
        #match jets
        matched = ~ak.any(ak.is_none(dijetEvents.FatJet.matched_gen, axis = -1), axis = -1)
        print("Length of matched ", len(matched))
        # NEED TO MAKE DELTA R MATCHING FOR GEN TO FIND MISSES
        #matched_gen = ~ak.is_none(DNE)
        
        
        # fakes are events reconstructed but does not exist in MC
        fakes = dijetEvents.FatJet[ak.any(ak.is_none(dijetEvents.FatJet.matched_gen, axis = -1), axis = -1)]
        print("Number of fake jets ", len(fakes))
        
        #miss = jet in MC but not reconstructed
        
        #misses = genJets[ak.is_none(DNE)]
        
        dijetEvents = dijetEvents[matched]
        print("Number of matched dijet events", len(dijetEvents))
        print("Check for none values", ak.any(ak.is_none(dijetEvents, axis = -1)))
        
        jet = dijetEvents.FatJet[:,:2]
        genjet = dijetEvents.GenJetAK8[:,:2]
        
        jet1 = dijetEvents.FatJet[:,0]
        jet2 = dijetEvents.FatJet[:,1]
        genjet1 = dijetEvents.GenJetAK8[:,0]
        genjet2 = dijetEvents.GenJetAK8[:,1]
        
        #flavour --> 21 is gluon
        
        jet2_g     = jet2[np.abs(genjet2.partonFlavour) == 21]
        jet2_uds   = jet2[np.abs(genjet2.partonFlavour) < 4]
        jet2_c     = jet2[np.abs(genjet2.partonFlavour) == 4]
        jet2_b     = jet2[np.abs(genjet2.partonFlavour) == 5]
        jet2_other = jet2[(np.abs(genjet2.partonFlavour) > 5) & (np.abs(genjet2.partonFlavour) != 21)]
        
        jet1_g     = jet1[np.abs(genjet1.partonFlavour) == 21]
        jet1_uds   = jet1[np.abs(genjet1.partonFlavour) < 4]
        jet1_c     = jet1[np.abs(genjet1.partonFlavour) == 4]
        jet1_b     = jet1[np.abs(genjet1.partonFlavour) == 5]
        jet1_other = jet1[(np.abs(genjet1.partonFlavour) > 5) & (np.abs(genjet1.partonFlavour) != 21)]
        
        #cant do jet_g = jet[jet.partonFlavor == 21] bc this would select events where either jet1 or jet2 is a gluon and keep both jets
        #jet_g = ak.concatenate((jet1_g, jet2_g), axis = 0)
#        jet_cat = ak.concatenate((jet1, jet2), axis = 0)
#        print("check that total jet cat would work same way. length of concat'd jets", len(ak.concatenate((jet1, jet2), axis = 0)))
#        print(" and flattened jets ",  len(ak.flatten(dijetEvents.FatJet[:,:2])), "\n")
        
        #make central and forward categories instead of jet1 jet2
#         print("Number of gluon jets = ", len(jet1_g)+len(jet2_g), " or ", len(jet_g))
        
        
        out['jet_mass'].fill(jetNumb = "jet1", partonFlav = "Gluon",  mass = jet1_g.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet1", partonFlav = "UDS",    mass = jet1_uds.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet1", partonFlav = "Charm",  mass = jet1_c.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet1", partonFlav = "Bottom", mass = jet1_b.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet1", partonFlav = "Other",  mass = jet1_other.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet2", partonFlav = "Gluon",  mass = jet2_g.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet2", partonFlav = "UDS",    mass = jet2_uds.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet2", partonFlav = "Charm",  mass = jet2_c.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet2", partonFlav = "Bottom", mass = jet2_b.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_mass'].fill(jetNumb = "jet2", partonFlav = "Other",  mass = jet2_other.mass,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        
        out['jet_pt'].fill(jetNumb = "jet1", partonFlav = "Gluon",  pt = jet1_g.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet1", partonFlav = "UDS",    pt = jet1_uds.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet1", partonFlav = "Charm",  pt = jet1_c.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet1", partonFlav = "Bottom", pt = jet1_b.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet1", partonFlav = "Other",  pt = jet1_other.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet2", partonFlav = "Gluon",  pt = jet2_g.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet2", partonFlav = "UDS",    pt = jet2_uds.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet2", partonFlav = "Charm",  pt = jet2_c.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet2", partonFlav = "Bottom", pt = jet2_b.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_pt'].fill(jetNumb = "jet2", partonFlav = "Other",  pt = jet2_other.pt,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )

        out['jet_eta'].fill(jetNumb = "jet1", partonFlav = "Gluon",  eta = jet1_g.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet1", partonFlav = "UDS",    eta = jet1_uds.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet1", partonFlav = "Charm",  eta = jet1_c.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet1", partonFlav = "Bottom", eta = jet1_b.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet1", partonFlav = "Other",  eta = jet1_other.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet2", partonFlav = "Gluon",  eta = jet2_g.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet2", partonFlav = "UDS",    eta = jet2_uds.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet2", partonFlav = "Charm",  eta = jet2_c.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet2", partonFlav = "Bottom", eta = jet2_b.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )
        out['jet_eta'].fill(jetNumb = "jet2", partonFlav = "Other",  eta = jet2_other.eta,
                             #weight = trijetEvents.Generator.weight[matched1]
                            )

        #NOTE --> need gen sd mass eventually --> recluster :( 
#         out['genjet_mass'].fill(
#             mass=ak.flatten(genjet.mass),
#             #weight=ak.flatten(dijetEvents.Generator.weight[~ak.is_none(dijetEvents.FatJet[:,0].matched_gen)])
#         )
            
#         out['weights'].fill(
#             dataset=events.metadata["dataset"],
#             weights=dijetEvents.Generator.weight,
#         )     
#         out['btag'].fill(
#             partonFlav = "Bottom",
#             bdisc=ak.flatten(jet_b.btagCSVV2),
#         )
        out['cutflow']['chunks'] += 1
        out['cutflow']['nGluonJets'] += (len(jet[np.abs(genjet.partonFlavour) == 21]))
        return out
    
    def postprocess(self, accumulator):
        return accumulator

In [3]:
tstart = time.time()
samples = {
    "QCD": files
}

result = processor.run_uproot_job(
    samples,
    "Events",
    DijetHists(),
    processor.futures_executor,
    {"schema": NanoAODSchema},
)
elapsed = time.time() - tstart
print(result)
print("Time taken to run over samples ", elapsed)
util.save(result, 'coffeaOutput/DiJet_pT30_eta25_result.coffea')

/opt/conda/lib/python3.8/site-packages/coffea/hist/hist_tools.py:850: RuntimeWarning: Allocating a large (>10M bin) histogram!
  warnings.warn("Allocating a large (>10M bin) histogram!", RuntimeWarning)


Output()

Output()

Initial # of dijet events  15000  and gen dijet events  15000 

# of dijet events after eta cut 14705  and gen dijet events  14705 

Length of matched  14705
Number of fake jets  19
Number of matched dijet events 14686


concurrent.futures.process._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/coffea/processor/executor.py", line 1542, in _work_function
    out = processor_instance.process(events)
  File "/tmp/ipykernel_20840/4065544809.py", line 109, in process
    print("Check for none values", ak.any(ak.is_none(dijetEvents, axis = -1)))
  File "/opt/conda/lib/python3.8/site-packages/awkward/operations/structure.py", line 2923, in is_none
    out = transform_outer(layout, 1, axis)
  File "/opt/conda/lib/python3.8/site-packages/awkward/operations/structure.py", line 2913, in transform_outer
    posaxis = layout.axis_wrap_if_negative(posaxis)
ValueError: axis=-1 exceeds the depth (1) of at least one record field (or union possibility) of this array

(https://github.com/scikit-hep/awkward-1.0/blob/1.8.0/src/libawkward/Content.cpp#L1743)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File

Initial # of dijet events  21000  and gen dijet events  21000 

# of dijet events after eta cut 20416  and gen dijet events  20416 

Length of matched  20416
Number of fake jets  27
Number of matched dijet events 20389
Initial # of dijet events  110998  and gen dijet events  110998 

# of dijet events after eta cut 107045  and gen dijet events  107045 

Length of matched  107045
Number of fake jets  132
Number of matched dijet events 106913
Initial # of dijet events  111997  and gen dijet events  111997 

# of dijet events after eta cut 107358  and gen dijet events  107358 

Length of matched  107358
Number of fake jets  107
Number of matched dijet events 107251
Initial # of dijet events  111999  and gen dijet events  111999 

# of dijet events after eta cut 107331  and gen dijet events  107331 

Length of matched  107331
Number of fake jets  117
Number of matched dijet events 107214
Initial # of dijet events  112000  and gen dijet events  112000 

# of dijet events after eta cut 10729

Exception: Failed processing file: WorkItem(dataset='QCD', filename='root://cmsxrootd.fnal.gov//store/mc/RunIISummer20UL17NanoAODv9/QCD_Pt_3200toInf_TuneCP5_13TeV_pythia8/NANOAODSIM/106X_mc2017_realistic_v9-v1/280000/B21BFDD8-B878-784C-AFB6-5CDE131F5C39.root', treename='Events', entrystart=0, entrystop=15000, fileuuid=b"\x0c\xc4X\xa2\xeb.\x11\xeb\x9b\x04{'\xbb\x94\xbe\xef", usermeta={})

In [ ]:
print()
import matplotlib.pyplot as plt


numerator = result['jet_pt'].integrate('partonFlav','Gluon').sum('jetNumb')
denominator = result['jet_pt'].sum('partonFlav').sum('jetNumb')
print(numerator, denominator)

numerator.label = r'$\epsilon$'
ax = hist.plotratio(
    num=numerator,
    denom=denominator,
    error_opts={'color': 'k', 'marker': '.'},
    unc='poisson-ratio'
)

# glu = result['jet_pt'].sum('jetNumb').values()[('Gluon',)]
# tot_parton = result['jet_pt'].sum('jetNumb').values()[('All',)]

# print("Check for nan values in tot_parton ", np.isnan(tot_parton))

# glu_frac = np.where(tot_parton > 0, np.divide(glu,tot_parton), 0)

# print(len(result['jet_pt'].axis('pt').centers()))
# print(result['jet_pt'].to_hist())

# print(glu_frac)

# plt.scatter(result['jet_pt'].axis('pt').centers(), glu_frac)

plt.xlabel("Pt (GeV)")
plt.ylabel("Gluon fraction")
plt.ylim([0.,1.0])


In [ ]:
numerator = result['jet_eta'].integrate('partonFlav','Gluon').sum("jetNumb")
denominator = result['jet_eta'].sum('partonFlav').sum("jetNumb")

print(numerator, denominator)

numerator.label = r'$\epsilon$'
ax = hist.plotratio(
    num=numerator,
    denom=denominator,
    error_opts={'color': 'k', 'marker': '.'},
    unc='poisson-ratio'
)

plt.xlabel("Eta < 1.7")
plt.ylabel("Gluon fraction")
plt.ylim([0.,1.0])


In [ ]:
#sum entries in bins --> plot as y-axis

# hist.plot1d(result['pt_jet2'], ax = axs[1], stack = True)
# axs[1].set_xlabel("Jet2 pt (GeV)")
# hist.plot1d(result['pt_jet1'], ax = axs[2], stack = True)
# axs[2].set_xlabel("Leading jet pt (GeV)")

In [ ]:
#plot pt, eta, phi, fake rate, pt efficiency, gluon purity for difference selections with b tags for all jets 
#and each individual jet in the 3 jets